<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-merging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc`

## Download and Install `mergekit`

In [ ]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

fatal: destination path 'mergekit' already exists and is not an empty directory.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done


## Create the YAML Config File to Merge Models with SLERP

In [ ]:
import yaml

### Write Config Script

In [ ]:
# Set model name
MODEL_NAME = 'whats-up-llamas'

#### SLERP

In [ ]:
# Write YAML config string
yamlConfigSLERP = """
slices:
  - sources:
      - model: meta-llama/Meta-Llama-3-8B-Instruct
        layer_range: [0, 32]
      - model: codellama/CodeLlama-7b-Instruct-hf
        layer_range: [0, 32]
merge_method: slerp
base_model: codellama/CodeLlama-7b-Instruct-hf
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

#### Passthrough

In [ ]:
yamlConfigPassthrough = """
slices:
  - sources:
    - model: meta-llama/Meta-Llama-3-8B-Instruct
      layer_range: [0, 32]
  - sources:
    - model: codellama/CodeLlama-7b-Instruct-hf
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16

"""

*Note: If you were to do this locally, instead of putting in the models' card name under `model`, you would specify the path to the model you downloaded from huggingface.*

### Save Config Script

In [ ]:
# Save config string as a YAML file
with open('config.yaml', 'w', encoding="utf-8") as fout:
    fout.write(yamlConfigSLERP)

## Merge Models

In [ ]:
# Merge models
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle